# ARC Internship Case

Author: Robin Zamojsky

In [364]:
# Python version: 3.7.0
#!pip install pandas==1.1.2

In [365]:
import pandas as pd

In [366]:
# Create dataframes from csv and json files
demographics_df = pd.read_csv('demographics.csv')
referral_df = pd.read_csv('referral.csv')
transactions_df = pd.read_csv('transactions.csv')
sub_prices_df = pd.read_json('subscription_prices.json')

### Descriptive statistics of the datasets

In [367]:
print(demographics_df.dtypes)
demographics_df.describe(include='all')

cust_id             object
customer_gender     object
age_group           object
customer_country    object
dtype: object


,cust_id,customer_gender,age_group,customer_country
count,10366,10366,10366,10366
unique,10366,5,5,6
top,9278,Other,18-24,Sweden
freq,1,4460,2536,4061


In [368]:
print(referral_df.dtypes)
referral_df.describe(include='all')

id               object
referral_type    object
dtype: object


,id,referral_type
count,19141,10996
unique,10366,7
top,1187,facebook
freq,6,3023


In [369]:
print(transactions_df.dtypes)
transactions_df.describe(include='all')

cust_id               int64
transaction_type     object
transaction_date     object
subscription_type    object
dtype: object


,cust_id,transaction_type,transaction_date,subscription_type
count,19141.000000,19141,19141,19141
unique,NaN,5,36,5
top,NaN,initial,2022-11-01,BASIC
freq,NaN,10366,764,7058
mean,5179.489003,NaN,NaN,NaN
std,2983.089553,NaN,NaN,NaN
min,1.000000,NaN,NaN,NaN
25%,2604.000000,NaN,NaN,NaN
50%,5154.000000,NaN,NaN,NaN
75%,7766.000000,NaN,NaN,NaN


In [370]:
print(sub_prices_df.dtypes)
sub_prices_df.describe(include='all')

BASIC    int64
PRO      int64
MAX      int64
dtype: object


,BASIC,PRO,MAX
count,3.0,3.0,3.0
mean,43.0,75.0,109.0
std,10.0,10.0,10.0
min,33.0,65.0,99.0
25%,38.0,70.0,104.0
50%,43.0,75.0,109.0
75%,48.0,80.0,114.0
max,53.0,85.0,119.0


### Data wrangling

In [371]:
# Remove string characters from cust_id column
demographics_df['cust_id'] = demographics_df['cust_id'].replace(r'[^0-9]+', '', regex=True)

# Convert cust_id column to int64
demographics_df['cust_id'] = demographics_df['cust_id'].astype('int64')

In [372]:
# Merge demographics_df and transactions_df on cust_id
df = pd.merge(demographics_df, transactions_df, on='cust_id', how='outer')

# Convert cust_id column to int64
df['cust_id'] = df['cust_id'].astype('int64')

df.describe(include='all')


,cust_id,customer_gender,age_group,customer_country,transaction_type,transaction_date,subscription_type
count,19141.000000,19141,19141,19141,19141,19141,19141
unique,NaN,5,5,6,5,36,5
top,NaN,Other,18-24,Sweden,initial,2022-11-01,BASIC
freq,NaN,8056,4646,7091,10366,764,7058
mean,5179.489003,NaN,NaN,NaN,NaN,NaN,NaN
std,2983.089553,NaN,NaN,NaN,NaN,NaN,NaN
min,1.000000,NaN,NaN,NaN,NaN,NaN,NaN
25%,2604.000000,NaN,NaN,NaN,NaN,NaN,NaN
50%,5154.000000,NaN,NaN,NaN,NaN,NaN,NaN
75%,7766.000000,NaN,NaN,NaN,NaN,NaN,NaN


In [373]:
df.head()

,cust_id,customer_gender,age_group,customer_country,transaction_type,transaction_date,subscription_type
0,1,Male,35-44,Denmark,initial,2020-09-01,BASIC
1,2,Other,18-24,Denmark,initial,2022-03-01,BASIC
2,3,Other,25-34,Norway,initial,2020-10-01,MAX
3,3,Other,25-34,Norway,REDUCTION,2022-02-01,BASIC
4,4,Male,18-24,Sweden,initial,2022-05-01,PRO


In [374]:
# Convert sub_prices_df to dictionary
sub_prices_dict = sub_prices_df.to_dict()

# Convert transaction_date column to datetime object
df['transaction_date'] = pd.to_datetime(df['transaction_date'])

# Extract year from transaction_date column and convert to numeric type
df['transaction_year'] = df['transaction_date'].dt.year

# Create function to retrieve subscription price from sub_prices_dict
def get_subscription_price(row):
    sub_type = row['subscription_type']
    sub_type = sub_type.upper()  # Convert to uppercase
    year = row['transaction_year']
    return sub_prices_dict[sub_type][year]

# Apply function to create subscription_price column
df['subscription_price'] = df.apply(get_subscription_price, axis=1)

df

,cust_id,customer_gender,age_group,customer_country,transaction_type,transaction_date,subscription_type,transaction_year,subscription_price
0,1,Male,35-44,Denmark,initial,2020-09-01,BASIC,2020,33
1,2,Other,18-24,Denmark,initial,2022-03-01,BASIC,2022,53
2,3,Other,25-34,Norway,initial,2020-10-01,MAX,2020,99
3,3,Other,25-34,Norway,REDUCTION,2022-02-01,BASIC,2022,53
4,4,Male,18-24,Sweden,initial,2022-05-01,PRO,2022,85
...,...,...,...,...,...,...,...,...,...
19136,10364,Female,45-54,Norway,UPGRADE,2022-09-01,MAX,2022,119
19137,10365,Male,55-65,Norway,initial,2021-05-01,PRO,2021,75
19138,10365,Male,55-65,Norway,REDUCTION,2021-07-01,BASIC,2021,43
19139,10366,Male,18-24,Denmark,initial,2021-01-01,BASIC,2021,43


In [375]:

# Remove string characters from id column
referral_df['id'] = referral_df['id'].replace(r'[^0-9]+', '', regex=True)

# Convert id column to int64
referral_df['id'] = referral_df['id'].astype('int64')

# Drop duplicate rows
referral_df.drop_duplicates(inplace=True)

# Rename id column to referrer_id
referral_df.rename(columns={'id': 'referrer_id'}, inplace=True)

referral_df


,referrer_id,referral_type
0,1,NaN
1,2,NaN
2,3,facebook
3,4,Display
4,5,NaN
...,...,...
10361,10362,facebook
10362,10363,NaN
10363,10364,NaN
10364,10365,Google Ads


In [376]:
# Add columns from referral_df to df with NaN values
referral_df = referral_df.assign(cust_id=pd.np.nan, customer_gender=pd.np.nan, age_group=pd.np.nan, customer_country=pd.np.nan, transaction_type=pd.np.nan, transaction_date=pd.np.nan, subscription_type=pd.np.nan)

# Concatenate the two dataframes
df = pd.concat([df, referral_df])

C:\Users\Robin\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:2: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead
  


In [377]:
# Save the final dataframe to a csv file
df.to_csv('compiled_data.csv', index_label='id')

### Conclusion

After exploring the datasets provided by the client, I have found that the datasets contain relevant and useful information for further analysis. The demographics.csv dataset contains customer attributes such as gender, age group, and country, which will be helpful in understanding the customer base. The transactions.csv dataset contains transaction information such as the type of transaction, transaction date, and subscription type, which will be useful in understanding customer behavior and subscription patterns. The subscription_prices.json file contains the subscription price points, which will be necessary for calculating revenue and understanding pricing strategies.

However, the referrals.csv dataset was unclear on what the id column represented. It was not clear if the id column was the same as the cust_id in the demographics and transactions datasets. Due to the ambiguity, the id column was treated as a separate identifier and not as the cust_id. Some minor data cleaning was performed on the referrals dataset before it was concatenated to the main dataset as a separate csv file.

Overall, the datasets provided by the client contain good information for further analysis. Although, further clarification on the referrals dataset would be helpful to better integrate the data into the analysis